# Д/З 1: Извлечение ключевых слов
### Выполнила Елизавета Клыкова, БКЛ181
#### Пункт 1. Создание мини-корпуса (1 балл)
*Подготовить мини-корпус (не меньше 4 текстов, примерный общий объём - 3-5 тысяч токенов) с разметкой ключевых слов. Предполагается, что вы найдете источник текстов, в котором уже выделены ключевые слова. Укажите источник корпуса и опишите, в каком виде там были представлены ключевые слова.*

Источник корпуса -- новостной сайт [vesti.ru](https://www.vesti.ru/news). Каждая новость имеет поле с тегами (расположено внизу после текста новости). Как правило, теги отражают не только общую тему, но и детали; обычно в их числе содержатся имена собственные, топонимы и т.д., реально встретившиеся в тексте. Таким образом, теги являются именно ключевыми словами, а не просто метками темы.

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import requests
import time
import nltk
import RAKE
import yake
import re
import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from sklearn.feature_extraction.text import TfidfVectorizer
from statistics import mean
from summa import keywords
from tqdm.auto import tqdm
session = requests.session()

In [3]:
def get_article_links():
    # загружаем страницу
    browser = webdriver.Chrome()
    page = browser.get('https://www.vesti.ru/news')

    # листаем вниз, чтобы прогрузилось больше статей
    elem = browser.find_element(By.TAG_NAME, 'body')
    for i in range(10):
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.2)

    # получаем ссылки на статьи
    link_list = browser.find_elements(By.XPATH, "//div[@class='list__item']/a")
    links = [art.get_attribute('href') for art in link_list]

    return links

In [4]:
def get_article_info(link):
    page = session.get(link).text
    soup = BeautifulSoup(page, 'html.parser')

    title = soup.find('title').text.strip()
    text = soup.find('div', {'class': 'article__text'}).text.strip()

    tags = []
    tags_list = soup.find('div', {'class': 'tags'}).find_all(
        'a', {'class': 'tags__item'})
    for t in tags_list:
        tag = t.text.strip()
        tags.append(tag)

    return title, text, ', '.join(tags)

In [5]:
def get_all_info():
    links = get_article_links()
    titles, texts, tag_lists = [], [], []

    for link in tqdm(links):
        title, text, tags = get_article_info(link)
        titles.append(title)
        texts.append(text)
        tag_lists.append(tags)

    return links, titles, texts, tag_lists

Выкачиваем статьи. Тексты получены за 18:35 3-го ноября 2021 г.

In [6]:
def get_news_corpus():

    # проверяем, существует ли готовый корпус
    if os.path.exists('mini_news_corpus.tsv'):
        news_df = pd.read_csv('mini_news_corpus.tsv', sep='\t')

    else:
        session = requests.session()
        links, titles, texts, tag_lists = get_all_info()
        clean_texts = [re.sub('((<|&.*?lt;).+?(>|&.*?gt;))', '', text)
                       for text in texts]
        lengths = [len(t.split()) for t in texts]
        news_df = pd.DataFrame({'title': titles,
                                'link': links,
                                'text': clean_texts,
                                'text_len': lengths,
                                'auto_tags': tag_lists})
        news_df.to_csv('mini_news_corpus.tsv', sep='\t', index=False)

    return news_df

In [7]:
news_df = get_news_corpus()

Проверяем, сколько получилось текстов и какова их суммарная длина.

In [8]:
lengths = news_df['text_len'].tolist()
corpus_stats = 'Всего текстов: {}, общее число токенов: {}.'.format(
    len(lengths), sum(lengths))

corpus_stats

'Всего текстов: 40, общее число токенов: 6698.'

Сразу лемматизируем тексты корпуса.

In [9]:
m = MorphAnalyzer()

In [10]:
def normalize_text(text, no_punct=False):

    # токенизация nltk (она точнее, чем у пайморфи, и не дробит числа)
    tokens = word_tokenize(text)

    # лемматизация пайморфи
    parsed = [m.parse(t)[0] for t in tokens]

    # если нужно, убираем пунктуацию
    # заодно избавляемся от странных неубиваемых кавычек
    if no_punct:
        lemmas = [w.normal_form for w in parsed
                  if 'PNCT' not in w.tag
                  and '``' not in w.normal_form]
    else:
        lemmas = [w.normal_form for w in parsed if '``' not in w.normal_form]

    return ' '.join(lemmas)

In [11]:
texts = news_df['text'].tolist()
lem_texts = [normalize_text(text, no_punct=True) for text in texts]
news_df['lem_text'] = lem_texts

#### Пункт 2. Создание ручной разметки (2 балла)
*Разметить ключевые слова самостоятельно. Оценить пересечение с имеющейся разметкой. Составить эталон разметки (например, пересечение или объединение вашей разметки и исходной).*

In [12]:
def get_manual_tags(news_df):

    # если в корпусе еще нет тегов, вводим через инпут
    if 'manual_tags' not in news_df.columns:

        texts = news_df['text'].tolist()
        manual_tag_lists = []
        for text in texts:
            print('\n**********\n\n', text, '\n\n**********\n')
            manual_tags = input('Введите ключевые слова: ')
            manual_tag_lists.append(manual_tags)
        news_df['manual_tags'] = manual_tag_lists

    return news_df

In [13]:
news_df = get_manual_tags(news_df)

Посмотрим, что получилось.

In [14]:
auto_tags = news_df['auto_tags'].tolist()
manual_tags = news_df['manual_tags'].tolist()

for i, tags in enumerate(auto_tags[0:5]):
    print('Исходные теги:', tags)
    print('Ручные теги:', manual_tags[i], '\n')

Исходные теги: общество, регионы, выходные, коронавирус, ограничение, пандемия, COVID-19, новости, ГТРК "Смоленск"
Ручные теги: коронавирус, ограничения, Смоленская область, локдаун, Курская область, Челябинская область, Брянская область, Новгородская область, нерабочие дни, выходные дни 

Исходные теги: происшествия, самолет, Белоруссия, крушение, Ан-12, Иркутск, новости
Ручные теги: крушение самолета, Ан-12, экипаж, ТАСС, Гродно, Иркутск, Пивовариха 

Исходные теги: происшествия, самолет, ЧП, новости, Новосибирск
Ручные теги: Boeing-747, аварийная посадка, самолет, ASL Airlines Belgium 

Исходные теги: экономика, Германия, Польша, Украина, газ, Северный поток - 2, новости
Ручные теги: газ, Газпром, Украина, Европа, Россия, Северный поток-2, Германия, Кублик, экономика 

Исходные теги: спорт, теннис, Карен Хачанов, новости, ATP-тур теннис
Ручные теги: Карен Хачанов, Париж, теннис, турнир, теннисный турнир, Андрей Рублев, Григор Димитров, Александр Зверев, Душан Лайович 



Можно собой гордиться -- получилось довольно точно. В случае с ручными тегами получился уклон в имена собственные, тогда как теги с сайта более общие. Попробуем взять перечение разметок.

In [15]:
auto_tgs = [set(tags.split(', ')) for tags in auto_tags]
manual_tgs = [set(tags.split(', ')) for tags in manual_tags]

In [16]:
for i, tags in enumerate(auto_tgs[0:5]):
    print(list(tags.intersection(manual_tgs[i])))

['коронавирус']
['Иркутск', 'Ан-12']
['самолет']
['Германия', 'Украина', 'газ', 'экономика']
['Карен Хачанов', 'теннис']


Пересения есть почти везде (лишь в 3 случаях из 40 пересечение пустое), но набора общих тегов не всегда достаточно для полноценного представления текста. Посмотрим на объединение разметок.

In [17]:
for i, tags in enumerate(auto_tgs[0:5]):
    print(sorted(list(tags.union(manual_tgs[i]))))

['COVID-19', 'Брянская область', 'ГТРК "Смоленск"', 'Курская область', 'Новгородская область', 'Смоленская область', 'Челябинская область', 'выходные', 'выходные дни', 'коронавирус', 'локдаун', 'нерабочие дни', 'новости', 'общество', 'ограничение', 'ограничения', 'пандемия', 'регионы']
['Ан-12', 'Белоруссия', 'Гродно', 'Иркутск', 'Пивовариха', 'ТАСС', 'крушение', 'крушение самолета', 'новости', 'происшествия', 'самолет', 'экипаж']
['ASL Airlines Belgium', 'Boeing-747', 'Новосибирск', 'ЧП', 'аварийная посадка', 'новости', 'происшествия', 'самолет']
['Газпром', 'Германия', 'Европа', 'Кублик', 'Польша', 'Россия', 'Северный поток - 2', 'Северный поток-2', 'Украина', 'газ', 'новости', 'экономика']
['ATP-тур теннис', 'Александр Зверев', 'Андрей Рублев', 'Григор Димитров', 'Душан Лайович', 'Карен Хачанов', 'Париж', 'новости', 'спорт', 'теннис', 'теннисный турнир', 'турнир']


Получается уже лучше, но есть несколько проблем:

1. Число существительных: так, в первом случае встречаются "ограничения" и "ограничение".
2. Повторения: "Роберт Дауни-мл" встречается в одном наборе с "Роберт Дауни-младший".
3. Мусорные теги: во всех наборах есть тег "новости". Зачем он, если мы и так знаем, что брали статьи с сайта новостей?
4. Мета-теги, которые не встречаются в самих текстах ("спорт", "общество"). Они могут относительно точно описывать содержание, но у тех алгоритмов, которые мы будем использовать, нет возможности изобрести эти теги, если их нет в текстах.

Первую проблему можно решить лемматизацией ключевых слов (тем более, это пригодится для сравнения автоматически выделенных слов и конструкций с эталоном), третью и четвертую -- выбрасыванием тегов, которые не встречаются в соответствующих текстах. Однако и без ручной очистки не обойтись.

In [18]:
tags_union = [tags.union(manual_tgs[i])
              for i, tags in enumerate(auto_tgs)]

In [19]:
final_tags = []

for tagset, text in list(zip(tags_union, lem_texts)):
    clean_tagset = []
    for tag in tagset:
        # лемматизируем, убирая пунктуацию
        lem_tag = normalize_text(tag, no_punct=True)
        # часть тегов заменяем (в текущем виде они не встречаются в текстах)
        clean_tag = lem_tag.replace(
                'atp-тур теннис', 'atp').replace(
                'wta-тур теннис', 'wta').replace(
                'сша/америка', 'америка').replace(
                'машина/автомобиль', 'автомобиль').replace(
                'смерть/кончин', 'смерть').replace(
                'александр звереть', 'александр зверев').replace(
                'министерство финансов/минфин', 'министерство финансов')
        # выбрасываем то, чего нет в тексте
        # в основном мета-теги "новости", "общество", "спорт"
        if clean_tag in text:
            clean_tagset.append(clean_tag)

    # убираем повторения, появившиеся после лемматизации
    final_tags.append(set(clean_tagset))

In [20]:
final_tags_str = [', '.join(tags) for tags in final_tags]
news_df['gold_tags'] = final_tags_str

In [21]:
for i in range(5):
    print(final_tags_str[i])

локдаун, курский область, смоленский область, новгородский область, выходной, коронавирус, нерабочий день, регион, брянский область, выходной день, челябинский область
крушение самолёт, экипаж, пивовариха, самолёт, иркутск, ан-12, гродно, тасс, крушение
asl airlines belgium, новосибирск, самолёт, аварийный посадка, boeing-747
газ, германия, кублик, европа, экономика, россия, газпром, северный поток-2, украина
александр зверев, андрей рублёв, турнир, григор димитров, теннис, душан лайович, париж, карен хачан, теннисный турнир


In [22]:
news_df.head()

,title,link,text,text_len,lem_text,auto_tags,manual_tags,gold_tags
0,Пять регионов в борьбе с COVID-19 продлили нер...,https://www.vesti.ru/article/2634849,В ряде российских регионов для сдерживания рас...,166,в ряд российский регион для сдерживание распро...,"общество, регионы, выходные, коронавирус, огра...","коронавирус, ограничения, Смоленская область, ...","локдаун, курский область, смоленский область, ..."
1,Крушение Ан-12 будет расследовать МАК,https://www.vesti.ru/article/2634804,Расследованием крушения самолета Ан-12 займетс...,79,расследование крушение самолёт ан-12 заняться ...,"происшествия, самолет, Белоруссия, крушение, А...","крушение самолета, Ан-12, экипаж, ТАСС, Гродно...","крушение самолёт, экипаж, пивовариха, самолёт,..."
2,В новосибирском Толмачево совершил экстренную ...,https://www.vesti.ru/article/2634873,Летевший из Бельгии в Китай грузовой Boeing-74...,44,лететь из бельгия в китай грузовой boeing-747 ...,"происшествия, самолет, ЧП, новости, Новосибирск","Boeing-747, аварийная посадка, самолет, ASL Ai...","asl airlines belgium, новосибирск, самолёт, ав..."
3,"СМИ Польши: Россия не заинтересована в ""Северн...",https://www.vesti.ru/finance/article/2634889,"Прокачка российского газа по трубопроводу ""Яма...",163,прокачка российский газ по трубопровод ямал ев...,"экономика, Германия, Польша, Украина, газ, Сев...","газ, Газпром, Украина, Европа, Россия, Северны...","газ, германия, кублик, европа, экономика, росс..."
4,Для Хачанова турнир в Париже закончен,https://www.vesti.ru/article/2634853,Карен Хачанов не смог выйти в третий круг прох...,91,карен хачан не смочь выйти в третий круг прохо...,"спорт, теннис, Карен Хачанов, новости, ATP-тур...","Карен Хачанов, Париж, теннис, турнир, теннисны...","александр зверев, андрей рублёв, турнир, григо..."


In [23]:
news_df.to_csv('mini_news_corpus.tsv', sep='\t', index=False)

#### Пункт 3. Извлечение ключевых слов (2 балла)
*Применить к этому корпусу 3 метода извлечения ключевых слов на выбор (RAKE, TextRank, tf-idf, OKAPI BM25, ...).*

#### RAKE

In [24]:
stop = stopwords.words('russian')
rake = RAKE.Rake(stop)

In [25]:
# тексты короткие --> minFrequency=1
# в эталоне есть конструкции из 3 слов --> maxWords=3
rake_kws = []
for text in lem_texts:
    kw_list = rake.run(text, maxWords=3, minFrequency=1)
    rake_kws.append(kw_list)

# обрезаем словосочетания с нулевой значимостью
rake_nonzero = [[kw for kw in kw_list if kw[-1] > 0] for kw_list in rake_kws]

In [26]:
rake_nonzero[0]

[('сохранение заработный плата', 9.0),
 ('удалённый формат взаимодействие', 9.0),
 ('глава курский область', 8.666666666666666),
 ('выходной день продлиться', 8.666666666666666),
 ('челябинский область отмечаться', 8.666666666666666),
 ('ряд российский регион', 8.0),
 ('регион нерабочий день', 7.166666666666666),
 ('нерабочий день', 5.166666666666666),
 ('новгородский область', 4.666666666666666),
 ('7 ноябрь ранее', 4.0),
 ('12 ноябрь включительно', 4.0),
 ('14 ноябрь включительно', 4.0),
 ('15 ноябрь кроме', 4.0),
 ('15 ноябрь продлить', 4.0),
 ('регион', 2.0),
 ('продление', 1.0),
 ('внести', 1.0),
 ('указ', 1.0),
 ('ввести', 1.0),
 ('28 октябрь', 1.0),
 ('неделя', 1.0)]

#### TextRank

In [27]:
textrank_kws = []
for text in lem_texts:
    kw_list = keywords.keywords(text, language='russian', ratio=0.15,
                                additional_stopwords=stop, scores=True)
    textrank_kws.append(kw_list)

In [28]:
textrank_kws[0]

[('область', 0.45644809380680473),
 ('продлиться', 0.2678215545264045),
 ('продлить режим', 0.2557232425480802),
 ('регион', 0.24197844692997414),
 ('ноябрь', 0.21250773634791137),
 ('власть', 0.17828231380378326),
 ('день', 0.1674290010638988),
 ('заработный', 0.15333775933879376),
 ('формат', 0.15333775933879362)]

#### TF-IDF

In [29]:
# векторизуем, выбрасывая стоп-слова и используя n-граммы
vectorizer = TfidfVectorizer(stop_words=stop, ngram_range=(1, 3))
vectors = vectorizer.fit_transform(lem_texts)

data = vectors.toarray()
words = np.array(vectorizer.get_feature_names())

In [30]:
# возьмем столько слов/n-грамм, сколько максимально бывает в эталоне
gold_kws_lem = [taglist.split(', ')
                for taglist in news_df['gold_tags'].tolist()]
n_max = max([len(i) for i in gold_kws_lem])

In [31]:
tfidf_kws = []

for doc in data:
    # сортируем индексы по убыванию значений
    sorted_row_idx = np.argsort(doc)[::-1][0:n_max]
    # получаем слова и n-граммы с такими индексами
    keywords = words[sorted_row_idx.ravel()]
    # получаем соответствующие значения
    kw_scores = doc[sorted_row_idx.ravel()]
    kw_with_scores = tuple(zip(keywords, kw_scores))
    tfidf_kws.append(kw_with_scores)

In [32]:
tfidf_kws[0]

(('нерабочий день', 0.2268795842568122),
 ('нерабочий', 0.2268795842568122),
 ('область', 0.2201740825189737),
 ('продлить', 0.20541331485732175),
 ('день', 0.18860865958395617),
 ('регион', 0.17961543678565908),
 ('режим', 0.16602652529802747),
 ('ноябрь', 0.15954403025705216),
 ('продлить режим нерабочий', 0.1370718658639827),
 ('продлить режим', 0.1370718658639827),
 ('режим нерабочий день', 0.12324798891439306),
 ('режим нерабочий', 0.12324798891439306),
 ('включительно', 0.1134397921284061))

#### YAKE
С YAKE у меня возникли некоторые проблемы: во-первых, если задать максимальный размер n-грамм = 3 и число выдаваемых слов = n_max (максимальное число слов в эталоне, у меня 13), то в выдаче всегда будут только сочетания из трех слов. Во-вторых, я вообще не понимаю, как работает параметр top))) потому что если задать top=30, в выдаче будет меньше 30 слов, зато размер n-грамм начнет варьироваться. Оставляю top=30, результат вроде бы выглядит адекватно. (Из-за неполного понимания работы алгоритма я решила, что лучше сделаю его четвертым в списке.)

In [33]:
# n -- максимальный размер n-грамм
# top -- число выделяемых слов (по идее, но на самом деле лотерея)
# dedupLim -- ограничение на дублирование слов в выдаче
# стоп-слова выбрасываются дефолтно и берутся из NLTK для указанного языка

yake_extractor = yake.KeywordExtractor(lan='ru', n=3,
                                       dedupLim=0.2, top=30, features=None)
yake_kws = []

for text in lem_texts:
    yake_kws.append(yake_extractor.extract_keywords(text))

In [34]:
# скоры обратно пропорциональны значимости
yake_kws[0]

[('продлить режим нерабочий', 0.0003236931906584315),
 ('островский подписать указ', 0.0009880119291171172),
 ('режим повышенный готовность', 0.0010022151249883932),
 ('ужесточать ограничительный мера', 0.0010159615912341729),
 ('высокий учебный заведение', 0.0010159615912341729),
 ('область губернатор регион', 0.002932689491805779),
 ('ноябрь', 0.009997807124087998),
 ('коронавирус ужесточать', 0.010017448676415503),
 ('ранее локдаун', 0.011405247838416658),
 ('указ', 0.048448977173929195),
 ('выходной', 0.048448977173929195),
 ('власть', 0.05661550740051128),
 ('ряд', 0.09958963134819915),
 ('мера', 0.09958963134819915),
 ('учиться', 0.09958963134819915),
 ('решение', 0.09958963134819915),
 ('жвачкина', 0.09958963134819915)]

#### Пункт 4. Составление шаблонов для ключевых слов (2 балла)
*Составить морфологические/синтаксические шаблоны для ключевых слов и фраз, выделить соответствующие им подстроки из корпуса (например, именные группы Adj+Noun). Применить эти фильтры к спискам ключевых слов.*

Какая информация вообще бывает?
* часть речи -- точно нужно использовать в шаблонах
* падеж и число -- пропадают при лемматизации, не помогут
* род -- пропадает у всего, кроме существительных, а для существительных это вряд ли важный критерий (было бы контринтуитивно считать, что сущ. м. р. чаще бывают ключевыми, чем сущ. ж. р.)
* одушевленность -- имеет смысл использовать, может быть важно
* вид и переходность глаголов -- не похоже, что должны влиять (можно поэкспериментировать, но мне лень)

In [35]:
def get_pattern_with_pos(tag):
    tag_pattern = []
    tag_parts = tag.split()
    for part in tag_parts:
        analysis = m.parse(part)[0].tag
        pos = analysis.POS
        if pos:
            tag_pattern.append(pos)
        # не теряем числа
        elif 'NUMB' in analysis:
            tag_pattern.append('NUMB')
        # учитываем иностранные названия
        elif 'LATN' in analysis:
            tag_pattern.append('LATN')
    return ' + '.join(tag_pattern)

In [36]:
def get_pattern_with_animacy(tag):
    tag_pattern = []
    tag_parts = tag.split()
    for part in tag_parts:
        analysis = m.parse(part)[0].tag
        pos = analysis.POS
        if pos:
            if pos == 'NOUN':
                anim = analysis.animacy
                pos = pos + ',' + anim
            tag_pattern.append(pos)
        # не теряем числа
        elif 'NUMB' in analysis:
            tag_pattern.append('NUMB')
        # учитываем иностранные названия
        elif 'LATN' in analysis:
            tag_pattern.append('LATN')
    return ' + '.join(tag_pattern)

In [37]:
gold_tags = [tgs.split(', ') for tgs in news_df['gold_tags'].tolist()]

In [38]:
pos_patterns, animacy_patterns = [], []
for tagset in gold_tags:
    for tag in tagset:
        pos_patterns.append(get_pattern_with_pos(tag))
        animacy_patterns.append(get_pattern_with_animacy(tag))

In [39]:
pos_patterns = set(pos_patterns)
pos_patterns.remove('')
animacy_patterns = set(animacy_patterns)
animacy_patterns.remove('')

In [40]:
pos_patterns

{'ADJF',
 'ADJF + ADJF + NOUN',
 'ADJF + NOUN',
 'ADJF + NOUN + NOUN',
 'ADJF + NOUN + NOUN + NOUN',
 'INFN',
 'LATN',
 'LATN + LATN',
 'LATN + LATN + LATN',
 'NOUN',
 'NOUN + ADJF',
 'NOUN + ADJF + NOUN',
 'NOUN + GRND',
 'NOUN + NOUN',
 'NOUN + NOUN + NOUN + NOUN',
 'NOUN + NUMB'}

In [41]:
animacy_patterns

{'ADJF',
 'ADJF + ADJF + NOUN,inan',
 'ADJF + NOUN,inan',
 'ADJF + NOUN,inan + NOUN,inan',
 'ADJF + NOUN,inan + NOUN,inan + NOUN,inan',
 'INFN',
 'LATN',
 'LATN + LATN',
 'LATN + LATN + LATN',
 'NOUN,anim',
 'NOUN,anim + ADJF',
 'NOUN,anim + GRND',
 'NOUN,anim + NOUN,anim',
 'NOUN,anim + NOUN,inan',
 'NOUN,inan',
 'NOUN,inan + ADJF + NOUN,inan',
 'NOUN,inan + NOUN,anim',
 'NOUN,inan + NOUN,anim + NOUN,anim + NOUN,anim',
 'NOUN,inan + NOUN,inan',
 'NOUN,inan + NUMB'}

In [42]:
def get_kws_without_scores(kws_with_scores):
    kws_without_scores = []
    for kw_set in kws_with_scores:
        kws_without_scores.append([kw[0] for kw in kw_set])
    return kws_without_scores

In [43]:
rake_full = get_kws_without_scores(rake_nonzero)
textrank_full = get_kws_without_scores(textrank_kws)
tfidf_full = get_kws_without_scores(tfidf_kws)
yake_full = get_kws_without_scores(yake_kws)

In [44]:
def filter_keywords_with_pos(keywords, patterns):
    filtered_kws = []
    for kw in keywords:
        kw_pattern = get_pattern_with_pos(kw)
        if kw_pattern in patterns:
            filtered_kws.append(kw)
    return filtered_kws

In [45]:
def filter_all_with_pos(kw_lists, patterns):
    filtered_kw_lists = []
    for kw_list in kw_lists:
        filtered_kw_lists.append(
            filter_keywords_with_pos(kw_list, patterns))
    return filtered_kw_lists

In [46]:
def filter_keywords_with_animacy(keywords, patterns):
    filtered_kws = []
    for kw in keywords:
        kw_pattern = get_pattern_with_animacy(kw)
        if kw_pattern in patterns:
            filtered_kws.append(kw)
    return filtered_kws

In [47]:
def filter_all_with_animacy(kw_lists, patterns):
    filtered_kw_lists = []
    for kw_list in kw_lists:
        filtered_kw_lists.append(
            filter_keywords_with_animacy(kw_list, patterns))
    return filtered_kw_lists

In [48]:
rake_with_pos = filter_all_with_pos(rake_full, pos_patterns)
textrank_with_pos = filter_all_with_pos(textrank_full, pos_patterns)
tfidf_with_pos = filter_all_with_pos(tfidf_full, pos_patterns)
yake_with_pos = filter_all_with_pos(yake_full, pos_patterns)

In [49]:
rake_with_animacy = filter_all_with_animacy(rake_full, animacy_patterns)
textrank_with_animacy = filter_all_with_animacy(textrank_full,
                                                animacy_patterns)
tfidf_with_animacy = filter_all_with_animacy(tfidf_full, animacy_patterns)
yake_with_animacy = filter_all_with_animacy(yake_full, animacy_patterns)

Выдача в целом похожая, не буду печатать, потому что там очень много. На результаты фильтрации с использованием части речи + одушевленности можно посмотреть в разделе 6 ниже.

#### Пункт 5. Оценка выделения ключевых слов (2 балла)
*Оценить точность, полноту, F-меру выбранных методов относительно эталона: с учётом морфосинтаксических шаблонов и без них.*

Приведем оценки для всех трех типов методов: без фильтрации, с фильтрацией только по частям речи и с фильтрацией по частям речи + одушевленности существительных.

In [50]:
def evaluate_set(gold, predicted):
    # у новости про Моргенштерна после фильтрации не осталось тегов
    if len(predicted) == 0:
        return 0, 0, 0

    gold = set(gold)
    predicted = set(predicted)
    precision = len(gold.intersection(predicted)) / len(predicted)
    recall = len(gold.intersection(predicted)) / len(gold)
    f1 = 0
    if precision + recall != 0:
        f1 = 2 * precision * recall / (precision + recall)

    return precision, recall, f1

In [51]:
def evaluate_all(gold_lists, predicted_lists):
    precisions, recalls, f1s = [], [], []

    for pair in list(zip(gold_lists, predicted_lists)):
        precision, recall, f1 = evaluate_set(pair[0], pair[1])
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)

    return [round(mean(precisions), 3),
            round(mean(recalls), 3),
            round(mean(f1s), 3)]

In [52]:
metrics = {'method': ['precision', 'recall', 'f1-score'],
           'RAKE_full': evaluate_all(gold_tags, rake_full),
           'RAKE_pos': evaluate_all(gold_tags, rake_with_pos),
           'RAKE_anim': evaluate_all(gold_tags, rake_with_animacy),
           'textrank_full': evaluate_all(gold_tags, textrank_full),
           'textrank_pos': evaluate_all(gold_tags, textrank_with_pos),
           'textrank_anim': evaluate_all(gold_tags, textrank_with_animacy),
           'tfidf_full': evaluate_all(gold_tags, tfidf_full),
           'tfidf_pos': evaluate_all(gold_tags, tfidf_with_pos),
           'tfidf_anim': evaluate_all(gold_tags, tfidf_with_animacy),
           'yake_full': evaluate_all(gold_tags, yake_full),
           'yake_pos': evaluate_all(gold_tags, yake_with_pos),
           'yake_anim': evaluate_all(gold_tags, yake_with_animacy),
           }

scores_df = pd.DataFrame(metrics).set_index('method').T

In [53]:
scores_df.style.highlight_max(
    color='palegreen', axis=0).highlight_min(
    color='lightsalmon', axis=0).format('{:.3f}')

method,precision,recall,f1-score
RAKE_full,0.063,0.151,0.083
RAKE_pos,0.113,0.148,0.116
RAKE_anim,0.117,0.148,0.118
textrank_full,0.100,0.128,0.107
textrank_pos,0.116,0.128,0.115
textrank_anim,0.118,0.128,0.115
tfidf_full,0.188,0.327,0.235
tfidf_pos,0.237,0.327,0.268
tfidf_anim,0.240,0.327,0.270
yake_full,0.065,0.148,0.088


(Примечание после сдачи д/з: там были красиво выделены минимальные и максимальные значения, но форматирование слетело при загрузке на гитхаб :(( )
Мы видим, что наиболее точным оказался метод TF-IDF (видимо, потому, что в корпусе достаточно разные тексты, не объединенные общей тематикой), наименее точным -- RAKE (но его различия с YAKE и TextRank не очень большие). Однако к результатам следует относиться с некоторой осторожностью. Во-первых, нужно понимать, что в случае с TF-IDF у нас была возможность выбрать параметры: сколько ключевых слов возвращать, что использовать при выделении ключевых слов (только слова / би-граммы / три-граммы). И если для RAKE существует по крайней мере возможность задать лимит по числу слов в конструкции, то TextRank это число определяет сам (поэтому в итоговом наборе есть конструкции из 4-5 слов). Аналогично, для TF-IDF количество ключевых слов и конструкций было жестко задано как максимальное число тегов в эталоне для одного текста, RAKE выбирал это число сам, а для TextRank была задана доля слов, которые можно выбрать как ключевые (т.е. число ключевых слов зависело от объема текста, а объем варьируется, что видно из датафрейма; при этом в эталоне такого разнообразия нет). Что касается YAKE, там вообще все плохо, начиная от несоответствия числа слов в выдаче заданному параметру top и заканчивая тем, что параметр top каким-то неочевидным способом влияет на число слов в ключевых конструкциях (???).

Заметно, что использование шаблонов улучшило метрики (причем более подробные шаблоны повлияли на качество сильнее, чем шаблоны, использующие только информацию о частях речи). Интересно, что влияние шаблонов вообще отсутствует в случае с YAKE (видимо, ему уже ничто не поможет))).

Посмотрим внимательнее на лучшие и худшие результаты в рамках отдельных методов.

In [54]:
rake_df = scores_df[0:3]
rake_df.style.highlight_max(
    color='palegreen', axis=0).highlight_min(
    color='lightsalmon', axis=0).format('{:.3f}')

method,precision,recall,f1-score
RAKE_full,0.063,0.151,0.083
RAKE_pos,0.113,0.148,0.116
RAKE_anim,0.117,0.148,0.118


In [55]:
textrank_df = scores_df[3:6]
textrank_df.style.highlight_max(
    color='palegreen', axis=0).highlight_min(
    color='lightsalmon', axis=0).format('{:.3f}')

method,precision,recall,f1-score
textrank_full,0.100,0.128,0.107
textrank_pos,0.116,0.128,0.115
textrank_anim,0.118,0.128,0.115


In [56]:
tfidf_df = scores_df[6:9]
tfidf_df.style.highlight_max(
    color='palegreen', axis=0).highlight_min(
    color='lightsalmon', axis=0).format('{:.3f}')

method,precision,recall,f1-score
tfidf_full,0.188,0.327,0.235
tfidf_pos,0.237,0.327,0.268
tfidf_anim,0.240,0.327,0.270


In [57]:
yake_df = scores_df[9:12]
yake_df.style.highlight_max(
    color='palegreen', axis=0).highlight_min(
    color='lightsalmon', axis=0).format('{:.3f}')

method,precision,recall,f1-score
yake_full,0.065,0.148,0.088
yake_pos,0.108,0.148,0.122
yake_anim,0.114,0.148,0.126


Очевидно, что в 3 случаях из 4 использование шаблонов дает бОльшие значения precision и f1, чем не-использование, причем более "продвинутые" шаблоны дают бОльший прирост качества (хоть и незначительно). Recall практически не меняется (лишь незначительно у RAKE).


#### Пункт 6. Описание ошибок и способы улучшения (1 балл)
*Описать ошибки автоматического выделения ключевых слов (что выделяется лишнее, что не выделяется); предложить свои методы решения этих проблем.*

##### Ошибки
У всех методов, особенно у RAKE, много ложноположительных (предсказанных тегов, которых нет в эталоне). Это связано в т.ч. с количеством выдаваемых ключевых слов (у RAKE и YAKE оно получилось в среднем самое большое, см. пункт 2 улучшений). У TF-IDF precision ожидаемо выше, потому что был задан жесткий порог по числу ключевых слов (можно было бы задать его как долю от слов текста и посмотреть, что получится, но, скорее всего, это не улучшило бы результаты, потому что число слов в эталоне практически не зависит от длины текста; я попробовала это сделать для TextRank в пунтке 2 улучшений, результат ожидаемый).

Полнота лучше всех у TF-IDF, т.е. в выделенные им ключевые слова и конструкции попало (относительно) много из того, что должно было попасть. Это несомненный плюс данного алгоритма, потому что у него размер выдачи был жестко ограничен, но это не помешало ему включить в нее достаточно много нужных слов. Из минусов можно отметить тот факт, что TF-IDF выделяет много повторов (*боровский, боровский птицефабрика, полигон боровский птицефабрика*), и на это никак нельзя повлиять (в отличие от, например, YAKE, где есть специальный отвечающий за это параметр.)

In [58]:
# простите за длинный принт, корпус большой :(
for i in range(10):
    gold = ', '.join(sorted(gold_tags[i]))
    rake = ', '.join(sorted(rake_with_animacy[i]))
    textrank = ', '.join(sorted(textrank_with_animacy[i]))
    tfidf = ', '.join(sorted(tfidf_with_animacy[i]))
    yake = ', '.join(sorted(yake_with_animacy[i]))
    print(
        'Текст №{}\nЭталон: {}\nRAKE: {}\nTextRank: {}\nTF-IDF: {}\nYAKE: {}\n'.format(
            i+1, gold, rake, textrank, tfidf, yake))

9:80: E501 line too long (87 > 79 characters)


Текст №1
Эталон: брянский область, выходной, выходной день, коронавирус, курский область, локдаун, нерабочий день, новгородский область, регион, смоленский область, челябинский область
RAKE: ввести, внести, неделя, нерабочий день, новгородский область, продление, регион, регион нерабочий день, ряд российский регион, сохранение заработный плата, удалённый формат взаимодействие, указ
TextRank: власть, день, заработный, ноябрь, область, продлиться, регион, формат
TF-IDF: день, нерабочий, нерабочий день, ноябрь, область, продлить, регион, режим, режим нерабочий день
YAKE: власть, высокий учебный заведение, выходной, жвачкина, мера, ноябрь, режим повышенный готовность, решение, ряд, указ, учиться

Текст №2
Эталон: ан-12, гродно, иркутск, крушение, крушение самолёт, пивовариха, самолёт, тасс, экипаж
RAKE: дождь, первый, приземлиться, смочь, ссылка
TextRank: белорусский, который, помешать, самолёт, сообщать, член
TF-IDF: белорусский, самолёт, село пивовариха, сообщать, член, член экипаж, экип

Видно, что среди автоматически выделенных ключевых слов, особенно в выдаче TextRank и YAKE, многовато глаголов (в эталоне они тоже встречались, раз появился такой шаблон, но намного реже: человеку более свойственно выделять в качестве ключевых именные группы). Возможное решение в пункте 3 улучшений.


##### Улучшения
1. Первое улучшение пришло мне в голову еще при выполнении 4 пункта, и я его уже реализовала: это использование дополнительной морфосинтаксической информации в шаблонах. В моем случае это только одушевленность, потому что остальная информация теряется при лемматизации (или вообще не имеет смысла, как род для существительных). На собранных данных получилось, что более подробный шаблон сработал лучше (хотя отличия минимальные).


2. Хочется как-то фильтровать ключевые слова, выдаваемые RAKE, потому что он почти всегда выдает больше, чем есть в эталоне, причем довольно значительно -- в 3-4 раза (на моем корпусе из 40 текстов обратная ситуация встретилась только трижды). То же самое для TextRank: доля выделяемых ключевых слов одинаковая для всего корпуса, а длина текстов в корпусе варьируется; при этом в эталоне число ключевых слов от 3 до 13, но чаще всего в диапазоне 6-9. **Апдейт**: я попробовала выставить для RAKE порог по частотности как долю от самого большого скора, получилось даже хуже, чем было))) Мне все еще хочется как-то фильтровать результаты, но я не очень представляю, как. **Апдейт 2**: для TextRank уменьшение ratio с дефолтного 0.2 до 0.15/0.1 дает прирост precision, но ухудшение recall (логично, меньше выдача --> меньше ложноположительных, но и меньше шанс, что в выдачу попадет все нужное). Я остановилась на ratio=0.15 как среднем варианте. См. картинку. ![Качество работы TextRank при ratio = 0.2, 0.15, 0.1](textrank.png)

In [59]:
# фильтрация выдачи RAKE по скору

rake_test = []
for art in rake_nonzero:
    new = []
    top_score = art[0][1]
    for kw in art:
        # пробовала долю от 0.05 до 0.3, чем меньше, тем лучше
        # т.е. лучше не фильтровать
        if kw[1] >= top_score * 0.2:
            new.append(kw[0])
    rake_test.append(new)

# для предсказаний без шаблонов
evaluate_all(gold_tags, rake_test)  # было 0.063, 0.151, 0.083

[0.042, 0.057, 0.045]

3. Можно попробовать учитывать не все шаблоны, а только самые частотные, но тогда нужно применять шаблоны и к самому эталону, иначе мы заведомо оставляем в эталоне что-то, что никак не может попасть в отфильтрованные предсказания (эта мысль была в беседе курса, я ее туда и писала))). Отчасти в этом есть смысл, потому что эталон не идеален (может быть, я один раз включила какую-то редкую конструкцию, а потом вытащила из предсказаний все n-граммы по этому шаблону, и получилось много лишнего).


4. Шаблоны далеки от идеала как минимум потому, что не идеальна разметка пайморфи. Например, некоторые слова он помечает как UNKN (не распознано), и непонятно, как относиться к этому тегу. В этой домашке принято волевое решение не считать UNKN за шаблон, что может быть не очень правильно, потому что в эталоне слова с таким тегом остаются, а в отфильтрованные предсказания попасть не могут, -- ровно та проблема, которая описана в абзаце выше. Все остальные варианты (NUMB и LATN) я учла, но на новых данных может вылезти еще что-то, что не попадает в поле POS и не будет учтено в шаблонах. Здесь возможное улучшение -- использовать другой парсе (об этом также в пункте 8).


5. При автоматическом выделении ключевых слов можно использовать дополнительную информацию, например, считать более важными те слова, которые встретились и в самом тексте, и в заголовке новости. Похожий подход можно использовать и при оценке результатов, сильнее штрафуя за не-выявление наиболее важных ключевых слов (более частотных / встретившихся в заголовке и т.д.)


6. Для TF-IDF хорошо бы использовать корпус побольше, иначе теряется смысл метода (но все-таки 40 текстов лучше, чем 4-5).


7. Выбранные метрики не очень подходят для оценки качества уже потому, что практически всегда имеет место несовпадение числа ключевых слов в эталоне и в выдаче разных алгоритмов, плюс опять-таки нужно учитывать важность тех или иных ключевых слов, а в нашем случае это невозможно уже потому, что эталон такой информации не содержит. В реальной задаче имело бы смысл использовать метрики, разработанные специально для выявления и ранжирования ключевых слов (Mean Reciprocal Rank, Mean Average Precision и т.д.)


8. Кое-где появились ошибки лемматизации, из-за которых выявились лишние ключевые слова ("газ" и "газа" в тексте №4) или выявилась другая форма ключевого слова (что, естественно, отразилось на метриках). Можно попробовать какой-то другой парсер (я хотела взять майстем, но библиотека на винде работает медленно, а с консольным мне было очень лень возиться).


9. И мое любимое -- доработать эталон :) Даже при всех произведенных ухищрениях в нем очень силен человеческий фактор. Если бы мы хотели использовать этот шаблон для какой-то реальной задачи, потребовалась бы независимая разметка ключевых слов несколькими экспертами с последующим формированием эталона на основе каким-то объективных критериев (например, сколько раз то или иное слово/конструкция встретилось в разметке разных людей).